In [1]:
import k3d
from scipy import ndimage
import numpy as np
import SimpleITK as sitk
import json
import nibabel as nib

In [10]:
class AsVolume:
    def __init__(self, img, annotations="empty"):
        # img orientation is "L", "I", "P". We only transform it for viewing purpose
        sitk_img = sitk.ReadImage(img)
        # img_data.shape = y,z,x
        sitk_img_data = sitk.GetArrayFromImage(sitk_img)
        # transpose. img_data.shape = z, y, x. .
        transp_sitk_img_data = np.transpose(sitk_img_data, [1, 0, 2])
        # z needs to be mirrored.
        trans_mat = np.array([[-1, 0, 0, 285],
                              [0, 1, 0, 0],
                              [0, 0, 1, 0],
                              [0, 0, 0, 1]])
        
        # inverted trans_mat has to be used according to scipy documentation
        inv_trans_mat = np.linalg.inv(trans_mat)
        
        new_sitk_img_data = ndimage.affine_transform(transp_sitk_img_data, inv_trans_mat)
        
        img_size = new_sitk_img_data.shape
        # we need to set the bounds to make sure, that the image shape stays the same
        bounds=np.array([0, img_size[2], 0, img_size[1], 0, img_size[0]])
        
        vol = k3d.volume(new_sitk_img_data.astype(np.float16),
                         color_range=[100, 300],
                         color_map=np.array(k3d.matplotlib_color_maps.gray,
                                            dtype=np.float32),
                         bounds = bounds)
        plot = k3d.plot()
        plot += vol
        
        if not annotations == "empty":
            file=open(annotations)
            annotation_system = json.load(file)
            annotation_shapes = annotation_system["annotations"][0]
            
            point_list=[]
            # create the points we want to use
            for points in annotation_shapes["points"]:
                # somehow Nora seems to use different notations for the array and 
                # the annotations so we need to transform the annotation coords: x, z, y -> z, y, x
                coords = [points["coords_vox"][1], points["coords_vox"][2], points["coords_vox"][0], 1]
                new_coords = np.matmul(trans_mat, coords)
                # k3d takes the array in the order z, y, x and changes it automatically to x, y, z
                # But we need to rearrange the coords of the annotations z, y, x -> x, y, z
                new_coords_clean= [new_coords[2], new_coords[1], new_coords[0]]
                item={"coords":new_coords_clean, "size":points["size"], "color":points["color"]}
                point_list.append(item)
            annos=[{"points":(point_list)}]
            
            for anno in annos[0]["points"]:
                # the color-giving for points is quite complicated with k3d. This will give it a red color.
                colormap = k3d.colormaps.matplotlib_color_maps.hsv
                red_color = k3d.helpers.map_colors(np.array(50), colormap)
                red_color = red_color.astype(np.uint32)
                new_point = k3d.points(positions=[anno["coords"][0], anno["coords"][1], anno["coords"][2]],
                                       point_size=anno["size"],
                                       colors=rand_color)
                plot += new_point
        else:
            pass
        
        plot.display()

In [11]:
img_filename = "..."
anno_filename = "..."
volume=AsVolume(img_filename, annotations=anno_filename)

Output()